<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Transportation%20Problem%20(with%20emission%20reduction%20strategies).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Das klassische Transportproblem

## Mathematisches Programm

**Notation:**
\begin{array}{ll}
A & \text{Set an Angebotsknoten}\\
N & \text{Set an Nachfrageknoten}\\
c_{ij}  & \text{Transportkosten zwischen Angebotsknoten $i \in A$ und Nachfrageknoten $j \in N$}\\
\end{array}
<br>

**Entscheidungsvariable:**
\begin{array}{ll}
x_{ij} & \text{Transportmenge zwischen Angebotsknoten $i \in A$ und Nachfrageknoten $j \in N$}\\
\end{array}
<br>

**Zielfunktion:** <br>
$
\begin{align}
  \begin{array}{llll}
    & \min \sum\limits_{i \in A} \sum\limits_{j \in N} c_{ij} \cdot x_{ij} & &~~~  (1) \\
  \end{array}
\end{align}
$
<br>

**Nebenbedingungen:**<br>
$
\begin{align}
  \begin{array}{llll}
    & a_i = \sum\limits_{j \in N} x_{ij} &,~ \forall~ i \in A &~~~ (2) \\
    & b_j = \sum\limits_{i \in A} x_{ij} &,~ \forall~ j \in N &~~~ (3) \\
    & x_{ij} \ge 0 &,~ \forall~ i \in A, j \in N &~~~ (4) \\
  \end{array}
\end{align}
$

## Implementierung

### Daten

In [ ]:
# Angebotsknoten
A = ['Sunnyvale', 'Dublin', 'Bankok']

# Angebotsmenge
a = {'Sunnyvale': 45, 'Dublin': 120, 'Bankok': 95}

# Nachfrageknoten
N = ['Amarillo', 'Teaneck', 'Chicago', 'Falls']

# Nachfragemenge
b = {'Amarillo': 80, 'Teaneck': 78, 'Chicago': 47, 'Falls': 55}

# Kosten
c = {
    'Sunnyvale': {'Amarillo': 250, 'Teaneck': 420, 'Chicago': 380, 'Falls': 280},
    'Dublin': {'Amarillo': 1280, 'Teaneck': 990, 'Chicago': 1440, 'Falls': 1520},
    'Bankok': {'Amarillo': 550, 'Teaneck': 1420, 'Chicago': 1660, 'Falls': 1730},
}

### Modell

In [ ]:
!pip install pulp
import pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 54.6 MB/s eta 0:00:00


In [ ]:
# Model
model = pulp.LpProblem(name='Transportproblem',
                       sense=pulp.constants.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts(name='x', indices=(A, N), lowBound=0, cat='Integer')

# (1) Objective
model += pulp.lpSum(c[i][j] * x[i][j] for i in A for j in N), '(1)'

# (2)
for i in A:
  model += a[i] == pulp.lpSum(x[i][j] for j in N), '(2)_' + str(i)

# (2)
for j in N:
  model += b[j] == pulp.lpSum(x[i][j] for i in A), '(3)_' + str(j)

# print(model)

### Solution

In [ ]:
# solve problem
model.solve()

# get status
print("Status:", pulp.LpStatus[model.status])

# get objective value
print('Objective value:', round(pulp.value(model.objective), 2))

# get value of decision variable u (position in tour of the nodes that are part of the tour)
for i in A:
  for j in N:
    if x[i][j].varValue > 0:
      print('Von ' + str(i) + ' nach ' + str(j) + ': ' + str(x[i][j].varValue) + ' ME')

Status: Optimal
Objective value: 219900.0
Von Sunnyvale nach Falls: 45.0 ME
Von Dublin nach Teaneck: 78.0 ME
Von Dublin nach Chicago: 42.0 ME
Von Bankok nach Amarillo: 80.0 ME
Von Bankok nach Chicago: 5.0 ME
Von Bankok nach Falls: 10.0 ME
